# WDPA-Intersection 

### Import Libraries

In [44]:
import pandas as pd
import geopandas as gpd
import yaml
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import plotly.express as px

### Load Shapefiles downloaded from WDPA website

In [45]:
# Load config.yaml
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

# Get base path from config
base_path = Path(config["wdpa_base_path"])

# Load the 3 individual shapefiles
WDPA_01 = gpd.read_file(base_path / "WDPA_Jun2025_Public_shp_0" / "WDPA_Jun2025_Public_shp-polygons.shp")
WDPA_02 = gpd.read_file(base_path / "WDPA_Jun2025_Public_shp_1" / "WDPA_Jun2025_Public_shp-polygons.shp")
WDPA_03 = gpd.read_file(base_path / "WDPA_Jun2025_Public_shp_2" / "WDPA_Jun2025_Public_shp-polygons.shp")


In [ ]:
# Merge the 3 datasets
merged_wdpa = gpd.GeoDataFrame(pd.concat([WDPA_01, WDPA_02, WDPA_03], ignore_index=True))
# Project 
merged_wdpa_3857 = merged_wdpa.to_crs(epsg=3857)


## Data exploration of WDPA

In [ ]:
merged_wdpa.head(3)

,WDPAID,WDPA_PID,PA_DEF,NAME,ORIG_NAME,DESIG,DESIG_ENG,DESIG_TYPE,IUCN_CAT,INT_CRIT,...,MANG_AUTH,MANG_PLAN,VERIF,METADATAID,SUB_LOC,PARENT_ISO,ISO3,SUPP_INFO,CONS_OBJ,geometry
0,1.0,1,1,Diamond Reef and Salt Fish Tail Reef,Diamond Reef,Marine Reserve,Marine Reserve,National,Ia,Not Applicable,...,Fisheries Division,Not Reported,State Verified,1807,AG-04,ATG,ATG,Not Applicable,Not Applicable,"POLYGON ((-61.82494 17.18497, -61.82497 17.184..."
1,2.0,2,1,Palaster Reef,Palaster Reef,Marine Reserve,Marine Reserve,National,Ia,Not Applicable,...,Fisheries Division,Not Reported,State Verified,1807,AG-10,ATG,ATG,Not Applicable,Not Applicable,"POLYGON ((-61.74007 17.52001, -61.77174 17.526..."
2,3.0,3,1,Laguna de los Pozuelos,Laguna de los Pozuelos,Monumento Natural,Nature Monument,National,III,Not Applicable,...,Administración de Parques Nacionales,Not Reported,State Verified,1935,AR-Y,ARG,ARG,Not Applicable,Not Applicable,"POLYGON ((-65.98955 -22.47423, -65.99441 -22.4..."


## Load points

In [ ]:
merged_wdpa.columns

Index(['WDPAID', 'WDPA_PID', 'PA_DEF', 'NAME', 'ORIG_NAME', 'DESIG',
       'DESIG_ENG', 'DESIG_TYPE', 'IUCN_CAT', 'INT_CRIT', 'MARINE',
       'REP_M_AREA', 'GIS_M_AREA', 'REP_AREA', 'GIS_AREA', 'NO_TAKE',
       'NO_TK_AREA', 'STATUS', 'STATUS_YR', 'GOV_TYPE', 'OWN_TYPE',
       'MANG_AUTH', 'MANG_PLAN', 'VERIF', 'METADATAID', 'SUB_LOC',
       'PARENT_ISO', 'ISO3', 'SUPP_INFO', 'CONS_OBJ', 'geometry'],
      dtype='object')

### Load points

In [ ]:
# Define sample power station locations (e.g in the Netherlands)
power_locations = {
    "Name" : ["Power Plant A", "Charging Station B", "Hydrogen Hub C", "Wind Site D", "Battery Bank E"],
    "energy_type" : ["Fossil", "Electric", "Hydrogen", "Wind", "Battery"],
    "operational": [True, True, False, True, False],
    "latitude": [52.310488, 52.093674, 51.933041, 53.221734, 51.433843],
    "longitude": [4.905643, 5.116298, 4.487232, 6.566158, 5.477312]
}
power_locations_df = pd.DataFrame(power_locations)

# Convert points to GeoDataFrame
power_locations_gdf = gpd.GeoDataFrame(
    power_locations_df,
    geometry=gpd.points_from_xy(power_locations_df.longitude, power_locations_df.latitude),
    crs="EPSG:4326"
)

In [ ]:
# Project to meters for buffering
power_locations = power_locations_gdf.to_crs(epsg=3857)
power_locations_gdf_buffer['geometry'] = power_locations.buffer(1000)  # 10km buffer

![Map Preview](map_preview01.png)

## Find intersections

In [ ]:
# Use spatial join function from GeoPandas. It combines two GeoDataFrames based on the spatial relationship between their geometries.
intersections = gpd.sjoin(power_locations_gdf_buffer, merged_wdpa_3857, how='inner', predicate='intersects')


In [ ]:
 # If a single point (or its buffer) intersects multiple polygons in the second GeoDataFrame, the spatial join will create one row per each intersection.
intersections
print(intersections.crs)

,Name,energy_type,operational,latitude,longitude,geometry,index_right,WDPAID,WDPA_PID,PA_DEF,...,OWN_TYPE,MANG_AUTH,MANG_PLAN,VERIF,METADATAID,SUB_LOC,PARENT_ISO,ISO3,SUPP_INFO,CONS_OBJ
0,Power Plant A,Fossil,True,52.310488,4.905643,"POLYGON ((547093.681 6856461.545, 547088.866 6...",197300,555638689.0,555638689,1,...,Not Reported,Ministry EZ RVO/GISCC,Not Reported,State Verified,2013,Not Reported,NLD,NLD,Not Applicable,Not Applicable
4,Battery Bank E,Battery,False,51.433843,5.477312,"POLYGON ((610731.583 6698397.313, 610726.767 6...",197303,555638692.0,555638692,1,...,Not Reported,Ministry EZ RVO/GISCC,Not Reported,State Verified,2013,Not Reported,NLD,NLD,Not Applicable,Not Applicable


## Finds non intersections

In [ ]:
# The spatial join (above) keeps the index from the left GeoDataFrame (shell_locations_gdf_buffer)
# intersections.index contains indices of all points that had at least one intersection
# We use ~ to invert the boolean mask, selecting only rows whose index is NOT in intersections

non_intersections = power_locations_gdf_buffer.loc[~power_locations_gdf_buffer.index.isin(intersections.index)]
print(non_intersections.crs)



## Visualize outputs

In [ ]:
# 1. Add an 'intersects' flag
intersections['intersects'] = True
non_intersections['intersects'] = False

# 2. Combine into a single GeoDataFrame
combined = pd.concat([intersections, non_intersections])

# 3. Plot with different colors based on 'intersects'
color_map = {True: 'red', False: 'purple'}
combined.explore(color=combined['intersects'].map(color_map), alpha=1)


/Users/olgavalenti/Workspace/gis/wdpa-intersection/venv/lib/python3.13/site-packages/geopandas/geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


![Map Preview](map_preview02.png)